</div>
<div align="left">
  <img src="img/abstract.png" width="400" alt="Funny little diagram">
  <p><em> Evolve nodes, evolve plans, and learn from the best performing ones.</em></p>
</div>
<div align="center">
</em></p>
</div>

#### Node Initialization (Refactoring ...)

In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-4qqnnwfdggvjba"
api_key = "rpa_EPOJED42G59S80Y6SKMCOI330EQU4JPPMKV2UD2W7j0uku"
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

Could not load vllm class, check CUDA support and GPU RAM size


In [2]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.evolnode import EvolNode
from methods.llm import get_groq_response, get_claude_response

# Code + Compilor Task
# mp = MetaPrompt("Search for age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.CODE)
# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

test_inputs = [c[0] for c in test_cases]

node = EvolNode(mp, None, None, get_response=get_endpoint_response, test_cases=test_cases) # setting manual test cases

node.evolve("i1", replace=True, batch_size=40, num_runs=2, print_summary=True) # Scale up batch size


input_dict = {"name": "Dilireba"}
output_dict = node(input_dict, max_attempts=6) # Batch Inference with vLLM

# node.get_response = get_groq_response # fast sequential inference
# output_dict = node(input_dict, max_attempts=6, batch_inference=False)
 
print("Output dict: ", output_dict)

Processing LLM queries: 100%|██████████| 40/40 [00:24<00:00,  1.61it/s]


 :: Total time elapsed: 24.91s, 0 errors


Processing LLM queries: 100%|██████████| 56/56 [00:40<00:00,  1.39it/s]


 :: Total time elapsed: 40.30s, 0 errors


Processing LLM queries: 0it [00:00, ?it/s]


 :: Total time elapsed: 0.00s, 0 errors
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 1.33
  🎯 Functional fitness: 0.00
  ⭐ Global fitness:     0.67
  🔄 Compiled solutions:        36
  ⏱️ Time breakdown:
     :: Query time: 23.32s
     :: Evolution time: 24.97s
     :: Evaluation time: 88.82s
     :: Total time: 137.12s


📊 Code 0: Fitness: 16.7%
--------------------------------------------------------------------------------
❌ Error Messages:
Input: {'name': 'Dilireba'}, prediction is not aligned with expected output, Expected: {'age': 32} Predicted: {'celebrities': [{'name': 'Dilireba', 'age': 29, 'nationality': 'Chinese', ' occupation': 'Actress, Singer'}, {'name': 'Fan Bingbing', 'age': 43, 'nationality': 'Chinese', 'occupation': 'Actress, Singer'}, {'name': 'Zhang Ziyi', 'age': 45, 'nationality': 'Chinese', 'occupation': 'Actress'}]}, Error message: Key age not found in prediction output


Input: {'name': 'ChengXiao'}, prediction is not aligned with expected output, Ex

Processing LLM queries: 100%|██████████| 6/6 [00:13<00:00,  2.25s/it]

 :: Total time elapsed: 13.48s, 0 errors
Output dict:  {'age': 32}


</div>
<div align="center">
  <img src="img/Project-Nirvana-evolve.gif" width="500" alt="Fourier reconstruction convergence">
  <p><em> Evolve a population of nodes. </em></p>
</div>

In [5]:
# Population building phase ... 
from methods.llm import get_groq_response, get_claude_response
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.population import Evolution

mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

evo = Evolution(pop_size=20, meta_prompt=mp, get_response=get_endpoint_response, 
                test_cases=test_cases, max_attempts=3, num_eval_runs=2,
                load=True)

strategies = ["m2"] # ["i1", "i1", "m2", "e2"]
evo.get_offspring(strategies)

evo.chat("How effective is the current evolution strategy? What improvement has it made in terms of fitness, and in terms of the implementation?",
         get_claude_response) 

# code-based check 
print(evo.population_info)

/opt/homebrew/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Processing LLM queries: 100%|██████████| 20/20 [00:16<00:00,  1.18it/s]


 :: Total time elapsed: 16.98s, 0 errors
Error occurred during API request: Function execution timed out (> 3 seconds)


KeyboardInterrupt: 


</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="900" alt="Fourier reconstruction convergence">
  <p><em> Plan, and evolve the plans. </em></p>
</div>


In [3]:
from methods.llm import get_claude_response, get_groq_response
from methods.diagram import visualize_plan_dict
from methods.meta_prompt import MetaPlan
from methods.evolnode import PlanNode


# Initialize PlanNode 
mp = MetaPlan("Get the age of celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"])
plan = PlanNode(mp, get_endpoint_response)

# i1 evolution of plan
plan_dicts, err_msg = plan.evolve_plan_dict(method="i1", batch_size=10) # Batch_size of 100 gives no slow-down

visualize_plan_dict(plan.plan_dict, plan.meta_prompt.task) # most simpliest plan

# Manual input on main-node test cases 
main_test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

is_success, err_msg = plan.spawn_test_cases(main_test_cases) #  pinned test cases generation
# plan.spawn_test_cases_majority(main_test_cases) # multi-agent test cases generation (need some benchmarking to compare quality)

 :: Evolving 10 plans in parallel...


/opt/homebrew/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Processing LLM queries: 100%|██████████| 10/10 [00:21<00:00,  2.14s/it]


 :: Total time elapsed: 21.45s, 0 errors
 :: Pseudo-code generated for each plan


Processing LLM queries: 100%|██████████| 10/10 [00:24<00:00,  2.40s/it]


 :: Total time elapsed: 24.02s, 0 errors
 :: Plan_dict generated for each plan


success: successfully compiled d2_output/plan_graph.d2 to d2_output/plan_graph.png in 126.367958ms
Processing LLM queries: 100%|██████████| 20/20 [00:21<00:00,  1.08s/it]

 :: Total time elapsed: 21.55s, 0 errors
Spawned 3 test cases for all sub-nodes


In [5]:
# plan.evolve_sub_nodes() # Completely stuck in the first call, debugging ... 

from methods.evolnode import EvolNode
from methods.meta_prompt import MetaPrompt, PromptMode

self = plan 

# 1. Did we skip existing nodes? Yes, those node has code & fitness and is skipped.
# 2. Did we make the name compatible with slightly off input?

for i, node_dict in enumerate(self.plan_dict["nodes"]):
    meta_prompt = MetaPrompt(
        task=node_dict["task"],
        func_name=node_dict["name"],
        inputs=node_dict["inputs"],
        outputs=node_dict["outputs"],
        input_types=node_dict["input_types"],
        output_types=node_dict["output_types"],
        mode=PromptMode((node_dict.get("mode", "code")).lower())
    )
    test_cases = self.test_cases_dict[node_dict["name"]]
    if "fitness" in node_dict and "code" in node_dict: 
        node = EvolNode(meta_prompt, node_dict["code"], node_dict["reasoning"], get_response=self.get_response, test_cases=test_cases, fitness=node_dict["fitness"])
    else:
        node = EvolNode(meta_prompt, None, None, get_response=self.get_response, test_cases=test_cases)
        print(f"🎲 :: Evolving {node.meta_prompt.func_name} ... ({i+1}/{len(self.plan_dict['nodes'])})")
        node.evolve("i1", replace=True, max_tries=2, num_runs=2, batch_size=20) # It's funny how 30+ sec could elapse before llm inference ... (collecting prompts ?? wtf is taking so long ??)
    self.nodes.append(node)

🎲 :: Evolving parse_birthdate ... (2/3)


/opt/homebrew/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Processing LLM queries: 100%|██████████| 20/20 [00:23<00:00,  1.18s/it]


 :: Total time elapsed: 23.70s, 0 errors
Error occurred during API request: Function execution timed out (> 3 seconds)
Error occurred during API request: Function execution timed out (> 3 seconds)
Error occurred during API request: Function execution timed out (> 3 seconds)
Error occurred during API request: Function execution timed out (> 3 seconds)
Error occurred during API request: Function execution timed out (> 3 seconds)
Error occurred during API request: Function execution timed out (> 3 seconds)
Error occurred during API request: Function execution timed out (> 3 seconds)
Error occurred during API request: Function execution timed out (> 3 seconds)
Error occurred during API request: Function execution timed out (> 3 seconds)
Error occurred during API request: Function execution timed out (> 3 seconds)


Processing LLM queries: 100%|██████████| 24/24 [00:38<00:00,  1.62s/it]


 :: Total time elapsed: 38.94s, 0 errors


Processing LLM queries: 100%|██████████| 14/14 [00:03<00:00,  3.62it/s]

 :: Total time elapsed: 3.87s, 0 errors


TypeError: 'float' object is not iterable

In [7]:
# node.evolve(method, replace=replace, max_tries=max_tries, num_runs=num_runs, batch_size=batch_size) # missing required input parameters: 'result' --> break down 

replace = True
method = "i1"
parents = []
batch_size = 20
num_runs = 2
max_tries = 2
self = node
feedback = ""

self.query_nodes(ignore_self=replace, self_func_name=self.meta_prompt.func_name) # look for relevant nodes in the library, go down

reasonings, codes = self._evolve(method, parents, batch_size=batch_size) # Issue #2. duplicate relevant functions after compilation, go down 

responses = self._get_evolve_response(method, parents, feedback, batch_size)

from methods.evolnode import parse_evol_response, compile_code_with_references

response = responses[0]
reasoning, code = parse_evol_response(response)
# code = compile_code_with_references(code, self.referrable_function_dict) # deal with node references | go down


/opt/homebrew/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Processing LLM queries: 100%|██████████| 20/20 [00:17<00:00,  1.12it/s]


 :: Total time elapsed: 17.81s, 0 errors


Processing LLM queries: 100%|██████████| 20/20 [00:44<00:00,  2.25s/it]

 :: Total time elapsed: 44.99s, 0 errors


ValueError: No code block found in the response.

In [9]:
print(response)

{To parse the Wikipedia page to extract the birthdate, we need to first identify the relevant section in the page, and then extract the birthdate information from that section. We will then generate a prompt to guide an AI in completing this task, specifying the input parameters, output format, and any additional requirements.}

{Based on the given tools and their fitness, we will use the `get_celeb_age` function to get the age of a celebrity, but since we are looking for birthdate, we will use this function in a way that will help us get birthdate. We will also use `search_google` function to search for the Wikipedia page of the celebrity if we don't have their Wikipedia page.}

def generate_prompt(wikipedia_page):
    # If we have the Wikipedia page, we will use get_celeb_age function
    if wikipedia_page:
        prompt = f"Given the Wikipedia page {{input: {wikipedia_page}}}, do extract the birthdate from the 'Early Life' section. Make sure the output is a json string in markdown 

In [102]:
print(code)

from datetime import datetime
from dateutil.relativedelta import relativedelta
from typing import Optional
def search_google(query: str) -> str:
    pass
def get_celeb_age(name: str) -> Optional[int]:
    pass
def parse_result(result: str) -> Optional[int]:
    """
    Parse the search result to extract the age of a celebrity.

    Args:
        result (str): The search result as a string.

    Returns:
        Optional[int]: The celebrity's age as an integer or None if not found.
    """
    search_result = search_google(name + ' birth date')
    birth_date_str = None
    for line in search_result.split('\n'):
        if 'birth date' in line.lower():
            birth_date_str = line.split(': ')[1]
            break
    if birth_date_str:
        birth_date = datetime.strptime(birth_date_str, '%B %d, %Y')
        current_year = datetime.now().year
        age = relativedelta(current_year, birth_date.year).years
        return age
    else:
        age_str = get_celeb_age(name)
       